<a href="https://colab.research.google.com/github/Segn11/zindi-financial-inclusion/blob/prediction2/financial_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_csv('/content/Train (6).csv')
test = pd.read_csv('/content/Test (5).csv')

In [ ]:
df.head()


,country,year,uniqueid,bank_account,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,uniqueid_1,Yes,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,uniqueid_2,No,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,uniqueid_3,Yes,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,uniqueid_4,No,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,uniqueid_5,No,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [33]:

df_id = df['uniqueid'].copy()
test_id = test["uniqueid"].copy()

# Prepare text data
X_df_raw = df.drop(columns=["uniqueid", "bank_account"]).copy()
X_te_raw = test.drop(columns=["uniqueid"]).copy()

# Combine for vectorization
all_data = pd.concat([X_df_raw, X_te_raw], axis=0).reset_index(drop=True)


In [ ]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [7]:
all_data.head()

,country,year,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,relationship_with_head,marital_status,education_level,job_type
0,Kenya,2018,Rural,Yes,3,24,Female,Spouse,Married/Living together,Secondary education,Self employed
1,Kenya,2018,Rural,No,5,70,Female,Head of Household,Widowed,No formal education,Government Dependent
2,Kenya,2018,Urban,Yes,5,26,Male,Other relative,Single/Never Married,Vocational/Specialised training,Self employed
3,Kenya,2018,Rural,Yes,5,34,Female,Head of Household,Married/Living together,Primary education,Formally employed Private
4,Kenya,2018,Urban,No,8,26,Male,Child,Single/Never Married,Primary education,Informally employed


In [34]:
cat_cols = all_data.select_dtypes(include="object").columns
cat_cols = list(cat_cols)
print(cat_cols)

['country', 'location_type', 'cellphone_access', 'gender_of_respondent', 'relationship_with_head', 'marital_status', 'education_level', 'job_type']


In [35]:
num_cols = all_data.select_dtypes(include="number").columns
num_cols = list(num_cols)
print(num_cols)

['year', 'household_size', 'age_of_respondent']


In [36]:
#feature engineering
all_data["is_large_family"] = (all_data["household_size"] > 5).astype(int)
all_data["household_per_age"] = all_data["household_size"] / (all_data["age_of_respondent"] + 1)
#all_data["has_phone_and_bank"] = ((all_data["cellphone_access"] == "Yes") & (y["bank_account"] == "Yes")).astype(int)
all_data["rural_and_low_education"] = ((all_data["location_type"] == "Rural") & (all_data["education_level"].isin(["No formal education", "Primary education"]))).astype(int)


In [37]:
edu_map = {
    "No formal education": 0,
    "Primary education": 1,
    "Secondary education": 2,
    "Tertiary education": 3
}
all_data["education_num"] = all_data["education_level"].map(edu_map)


In [38]:
all_data.isna().sum()

,0
country,0
year,0
location_type,0
cellphone_access,0
household_size,0
age_of_respondent,0
gender_of_respondent,0
relationship_with_head,0
marital_status,0
education_level,0


In [43]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33610 entries, 0 to 33609
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   country                  33610 non-null  object 
 1   location_type            33610 non-null  object 
 2   cellphone_access         33610 non-null  object 
 3   household_size           33610 non-null  float64
 4   age_of_respondent        33610 non-null  float64
 5   gender_of_respondent     33610 non-null  object 
 6   relationship_with_head   33610 non-null  object 
 7   marital_status           33610 non-null  object 
 8   education_level          33610 non-null  object 
 9   job_type                 33610 non-null  object 
 10  is_large_family          33610 non-null  float64
 11  household_per_age        33610 non-null  float64
 12  rural_and_low_education  33610 non-null  int64  
 13  education_num            33610 non-null  float64
dtypes: float64(5), int64(1

In [40]:
#handling missing values
all_data["education_num"].fillna(0, inplace=True)



/tmp/ipython-input-3121571800.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data["education_num"].fillna(0, inplace=True)


In [41]:
#feature selection
all_data.drop(columns=["year"], inplace=True)

In [42]:
num_cols = all_data.select_dtypes(include="number").columns
skewed = all_data[num_cols].apply(lambda x: x.dropna().skew()).abs()
skew_cols = skewed[skewed > 0.75].index
all_data[skew_cols] = np.log1p(all_data[skew_cols])

In [45]:
from sklearn.preprocessing import LabelEncoder
#float_array = all_data[["household_size", "age_of_respondent", "year"]].values.astype(float)

# List of categorical columns for One-Hot Encoding
cat_cols = [
    "relationship_with_head",
    "marital_status",
    "education_level",
    "job_type",
    "country",

]

# Keep only columns that exist in all_data
cat_cols = [col for col in cat_cols if col in all_data.columns]

# One-Hot Encoding
all_data = pd.get_dummies(all_data, columns=cat_cols, prefix_sep="_")

# Label encoding for small/binary categorical columns
le = LabelEncoder()
for col in ["location_type", "cellphone_access", "gender_of_respondent", "rural_and_low_education"]:
    if col in all_data.columns:
        all_data[col] = le.fit_transform(all_data[col].astype(str))


In [46]:
all_data.head()

,location_type,cellphone_access,household_size,age_of_respondent,gender_of_respondent,is_large_family,household_per_age,rural_and_low_education,education_num,relationship_with_head_Child,...,job_type_Government Dependent,job_type_Informally employed,job_type_No Income,job_type_Other Income,job_type_Remittance Dependent,job_type_Self employed,country_Kenya,country_Rwanda,country_Tanzania,country_Uganda
0,0,1,1.386294,3.218876,0,0.000000,0.113329,0,2.0,False,...,False,False,False,False,False,True,True,False,False,False
1,0,0,1.791759,4.262680,0,0.000000,0.068053,1,0.0,False,...,True,False,False,False,False,False,True,False,False,False
2,1,1,1.791759,3.295837,1,0.000000,0.169899,0,0.0,False,...,False,False,False,False,False,True,True,False,False,False
3,0,1,1.791759,3.555348,0,0.000000,0.133531,1,1.0,False,...,False,False,False,False,False,False,True,False,False,False
4,1,0,2.197225,3.295837,1,0.693147,0.259511,0,1.0,True,...,False,True,False,False,False,False,True,False,False,False


In [47]:
# If train['bank_account'] contains 'Yes'/'No'
y = df['bank_account'].map({'No': 0, 'Yes': 1})


In [48]:
n_train = X_df_raw.shape[0]
X_train = all_data.iloc[:n_train, :].copy()
X_test  = all_data.iloc[n_train:, :].copy()

In [49]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)



In [50]:
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score

model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_tr, y_tr)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2670, number of negative: 16149
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002550 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 18819, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.141878 -> initscore=-1.799780
[LightGBM] [Info] Start training from score -1.799780
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

LGBMClassifier(colsample_bytree=0.8, learning_rate=0.05, max_depth=7,
               n_estimators=500, random_state=42, subsample=0.8)

In [52]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.3 MB/s eta 0:00:00


In [53]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [54]:
# ---------------------------------------------
# 0️⃣ Imports
# ---------------------------------------------
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# ---------------------------------------------
# 1️⃣ Split Data (if not already done)
# ---------------------------------------------
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------------------------------------
# 2️⃣ Define Optuna Objective Function
# ---------------------------------------------
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 2.0),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "eval_metric": "AUC",
        "verbose": 0,
        "auto_class_weights": "Balanced",  # handles imbalance in target
    }

    model = CatBoostClassifier(**params)
    model.fit(
        X_tr,
        y_tr,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        use_best_model=True
    )

    preds = model.predict_proba(X_val)[:, 1]
    auc = roc_auc_score(y_val, preds)
    return auc

# ---------------------------------------------
# 3️⃣ Run Optuna Study
# ---------------------------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Increase trials for deeper tuning

# ---------------------------------------------
# 4️⃣ Train Final Model with Best Parameters
# ---------------------------------------------
best_params = study.best_params
print("✅ Best Parameters:", best_params)

model = CatBoostClassifier(**best_params)
model.fit(
    X_tr,
    y_tr,
    eval_set=(X_val, y_val),
    early_stopping_rounds=50,
    use_best_model=True,
)

# ---------------------------------------------
# 5️⃣ Evaluate on Validation Data
# ---------------------------------------------
preds = model.predict_proba(X_val)[:, 1]
val_mae = mean_absolute_error(y_val, preds)
auc = roc_auc_score(y_val, preds)

print(f"Validation MAE: {val_mae:.4f}")
print(f"Validation ROC AUC: {auc:.4f}")


[I 2025-10-20 06:54:21,587] A new study created in memory with name: no-name-892ef597-9bb9-432c-ab18-c7d1cdbe5e2a
[I 2025-10-20 06:54:22,577] Trial 0 finished with value: 0.8675436190857646 and parameters: {'iterations': 852, 'learning_rate': 0.2569156422220763, 'depth': 8, 'l2_leaf_reg': 0.7678973919171935, 'bagging_temperature': 0.3021051811802066, 'random_strength': 1.2336500057027242, 'border_count': 193}. Best is trial 0 with value: 0.8675436190857646.
[I 2025-10-20 06:54:25,552] Trial 1 finished with value: 0.8668430684342712 and parameters: {'iterations': 756, 'learning_rate': 0.12315897537229517, 'depth': 10, 'l2_leaf_reg': 0.09787070471093086, 'bagging_temperature': 0.037897960806332565, 'random_strength': 1.732472285112182, 'border_count': 189}. Best is trial 0 with value: 0.8675436190857646.
[I 2025-10-20 06:54:29,167] Trial 2 finished with value: 0.8733755631493171 and parameters: {'iterations': 651, 'learning_rate': 0.012361766737669248, 'depth': 7, 'l2_leaf_reg': 0.126397

✅ Best Parameters: {'iterations': 709, 'learning_rate': 0.027009572803330155, 'depth': 5, 'l2_leaf_reg': 8.484111218731163, 'bagging_temperature': 0.4183453802582977, 'random_strength': 1.5858811010257656, 'border_count': 128}
0:	learn: 0.6696120	test: 0.6697238	best: 0.6697238 (0)	total: 5.63ms	remaining: 3.99s
1:	learn: 0.6477270	test: 0.6479280	best: 0.6479280 (1)	total: 10.9ms	remaining: 3.85s
2:	learn: 0.6254848	test: 0.6255785	best: 0.6255785 (2)	total: 16.1ms	remaining: 3.79s
3:	learn: 0.6056380	test: 0.6056751	best: 0.6056751 (3)	total: 22ms	remaining: 3.88s
4:	learn: 0.5892607	test: 0.5889731	best: 0.5889731 (4)	total: 27.3ms	remaining: 3.85s
5:	learn: 0.5728463	test: 0.5722983	best: 0.5722983 (5)	total: 32.6ms	remaining: 3.82s
6:	learn: 0.5570894	test: 0.5566900	best: 0.5566900 (6)	total: 37.8ms	remaining: 3.79s
7:	learn: 0.5425157	test: 0.5420039	best: 0.5420039 (7)	total: 43ms	remaining: 3.77s
8:	learn: 0.5267305	test: 0.5262691	best: 0.5262691 (8)	total: 48.3ms	remaining: 

In [51]:
y_val_pred = model.predict(X_val)
y_val_prob = model.predict_proba(X_val)[:, 1]  # probability of class 1

# Since Zindi uses MAE
val_mae = mean_absolute_error(y_val, y_val_pred)
print("Validation MAE:", val_mae)

# Optional: ROC-AUC for additional insight
roc_auc = roc_auc_score(y_val, y_val_prob)
print("Validation ROC-AUC:", roc_auc)


Validation MAE: 0.112221041445271
Validation ROC-AUC: 0.8608213472696005


In [55]:
model.fit(X_train, y)  # train on all training data
test_preds = model.predict(X_test)




0:	learn: 0.6695508	total: 7.5ms	remaining: 5.31s
1:	learn: 0.6475899	total: 14.2ms	remaining: 5.01s
2:	learn: 0.6252824	total: 20ms	remaining: 4.71s
3:	learn: 0.6049991	total: 25.8ms	remaining: 4.55s
4:	learn: 0.5876596	total: 31.6ms	remaining: 4.45s
5:	learn: 0.5721932	total: 37.2ms	remaining: 4.35s
6:	learn: 0.5555718	total: 43.1ms	remaining: 4.32s
7:	learn: 0.5409836	total: 49.3ms	remaining: 4.32s
8:	learn: 0.5271812	total: 55.4ms	remaining: 4.31s
9:	learn: 0.5140931	total: 62.8ms	remaining: 4.39s
10:	learn: 0.5025121	total: 69.2ms	remaining: 4.39s
11:	learn: 0.4913291	total: 75.1ms	remaining: 4.36s
12:	learn: 0.4777070	total: 80.8ms	remaining: 4.33s
13:	learn: 0.4673888	total: 86.9ms	remaining: 4.31s
14:	learn: 0.4584842	total: 92.9ms	remaining: 4.3s
15:	learn: 0.4490489	total: 98.7ms	remaining: 4.27s
16:	learn: 0.4389304	total: 110ms	remaining: 4.47s
17:	learn: 0.4322131	total: 116ms	remaining: 4.46s
18:	learn: 0.4252544	total: 122ms	remaining: 4.43s
19:	learn: 0.4184185	total: 1

In [ ]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})

In [56]:
# test_id: Series of unique IDs
# test_country: Series of country names corresponding to test IDs
test_country = test["country"].copy()  # make sure you have this column

# Prepare submission
submission = pd.DataFrame({
    "unique_id": test_id.astype(str) + " x " + test_country.astype(str),
    "bank_account": test_preds
})

submission.to_csv("submission.csv", index=False)
print("✅ Submission saved!")


✅ Submission saved!


In [57]:
submission.head()

,unique_id,bank_account
0,uniqueid_6056 x Kenya,1
1,uniqueid_6060 x Kenya,1
2,uniqueid_6065 x Kenya,0
3,uniqueid_6072 x Kenya,0
4,uniqueid_6073 x Kenya,0


In [58]:
from google.colab import files
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>